In [2]:
import re
import json
import torch
import numpy as np
from tqdm import tqdm 
import itertools

def load_attr_dict(file):
    # 读取属性字典
    with open(file, 'r') as f:
        attr_dict = {}
        for attr, attrval_list in json.load(f).items():
            attrval_list = list(map(lambda x: x.split('='), attrval_list))
            attr_dict[attr] = list(itertools.chain.from_iterable(attrval_list))
    return attr_dict

def match_attrval(title, attr, attr_dict):
    # 在title中匹配属性值
    attrvals = "|".join(attr_dict[attr])
    ret = re.findall(attrvals, title)
    # return "{}{}".format(attr, ''.join(ret))
    return "{}".format(''.join(ret))  

# load attribute dict
attr_dict_file = "../data/attr_to_attrvals.json"
attr_dict = load_attr_dict(attr_dict_file)

In [26]:
attr_list = []
for item in attr_dict.values():
    attr_list+=item
len(attr_list), len(list(set(attr_list)))

(80, 78)

In [19]:
# remove years [fine]
fine_data = '../data/train_fine.txt'
new_fine_data = '../data/processed_data/train_fine_noyear.txt'

rets = []
i = 0
with open(fine_data, 'r') as f:
    for i, data in enumerate(tqdm(f)):
        data = json.loads(data)
        title = data['title']
        title = re.sub(r'\d{1,4}年','', title)
        data['title'] = title
        
        rets.append(json.dumps(data, ensure_ascii=False)+'\n')
        
with open(new_fine_data, 'w') as f:
    f.writelines(rets)

50000it [01:37, 512.80it/s]


In [31]:
# remove years and get attributes [coarse]
# 裤门襟和鞋带的属性有重合，所以需要额外的判断机制
coarse_data = '../data/train_coarse.txt'
new_coarse_data = '../data/processed_data/coarse_fine_noyear.txt'

rets = []
querys = attr_dict.keys()
i = 0
with open(coarse_data, 'r') as f:
    for i, data in enumerate(tqdm(f)):
        data = json.loads(data)
        title = data['title']
        title = re.sub(r'\d{1,4}年','', title)
        data['title'] = title
        
        if data['match']['图文'] == 1:
            for query in querys:
                values = attr_dict[query]
                if (query == '裤门襟') and ('裤' not in title):
                    continue
                if (query == '闭合方式') and ('裤' in title):
                    continue
                for value in values:
                    if value in title:
                        data['key_attr'][query] = value
                        data['match'][query] = 1
                    
        rets.append(json.dumps(data, ensure_ascii=False)+'\n')     
        
with open(new_coarse_data, 'w') as f:
    f.writelines(rets)

100000it [03:13, 516.96it/s]


In [33]:
a = '领形是V领圆领'
re.findall('V领|圆领',a)

['V领', '圆领']